# Project 06: SQL Database Queries with SQLite

## Overview
In this project, we explore how to interact with a SQLite database using Python. The focus is on writing SQL queries to extract, filter, join, and aggregate customer orders data.

## Objective
- Connect to a SQLite database using Python.
- Learn basic SQL commands: SELECT, WHERE, JOIN, GROUP BY.
- Retrieve meaningful insights from the database.
- Understand integration of SQL with Python scripts.


## Step 1: Import libraries  
We use `sqlite3` to connect and query SQLite databases, and `pandas` to handle data in tabular form.


In [1]:
import sqlite3
import pandas as pd

## Step 2: Database connection  
We connect to an existing SQLite database or create a new one if it doesn't exist.


In [5]:
conn = sqlite3.connect('customer_orders.db')
cursor = conn.cursor()

## Step 3: Create tables and insert sample data  
If no database exists, we create customers and orders tables, then add some example data for querying.


In [6]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    city TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    amount REAL,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
)
''')

cursor.execute("INSERT INTO customers (name, city) VALUES ('Alice', 'New York')")
cursor.execute("INSERT INTO customers (name, city) VALUES ('Bob', 'Los Angeles')")
cursor.execute("INSERT INTO orders (customer_id, order_date, amount) VALUES (1, '2023-01-10', 250.0)")
cursor.execute("INSERT INTO orders (customer_id, order_date, amount) VALUES (2, '2023-01-11', 450.5)")
conn.commit()


## Step 4: Query all customers  
Retrieve all customers from the database and display as a DataFrame.


In [8]:
customers_df = pd.read_sql_query("SELECT * FROM customers", conn)
customers_df


,customer_id,name,city
0,1,Alice,New York
1,2,Bob,Los Angeles


## Step 5: Filter customers by city  
Select customers who are based in New York.


In [9]:
ny_customers = pd.read_sql_query("SELECT * FROM customers WHERE city = 'New York'", conn)
ny_customers


,customer_id,name,city
0,1,Alice,New York


## Step 6: Join tables  
Get a combined view of orders along with customer names by joining the two tables.


In [12]:
orders_with_customers = pd.read_sql_query('''
SELECT orders.order_id, customers.name, orders.order_date, orders.amount
FROM orders
JOIN customers ON orders.customer_id = customers.customer_id
''', conn)
orders_with_customers


,order_id,name,order_date,amount
0,1,Alice,2023-01-10,250.0
1,2,Bob,2023-01-11,450.5


## Step 7: Aggregate data  
Calculate the total amount each customer has spent.


In [13]:
total_amounts = pd.read_sql_query('''
SELECT customers.name, SUM(orders.amount) as total_spent
FROM orders
JOIN customers ON orders.customer_id = customers.customer_id
GROUP BY customers.name
''', conn)
total_amounts


,name,total_spent
0,Alice,250.0
1,Bob,450.5


## Step 8: Close connection  
Always close the connection to the database when done.


In [14]:
conn.close()